In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet("news.parquet")

In [4]:
MISTRAL_API_KEY = ''

In [3]:
df

,rank,title,url,points,author,created_at,comments,partition_date
0,1,Simulating water over terrain,https://lisyarus.github.io/blog/posts/simulati...,387,ibobev,2025-02-06T14:15:28 1738851328,58,2025-02-06
1,2,Programming SDF animations of Rick and Morty,https://danielchasehooper.com/posts/code-anima...,505,LordNibbler,2025-02-06T03:30:34 1738812634,39,2025-02-06
2,3,Show HN: SQLite disk page explorer,https://github.com/QuadrupleA/sqlite-page-expl...,288,QuadrupleA,2025-02-06T18:40:30 1738867230,41,2025-02-06
3,4,Paper Apps,https://gladdendesign.com/collections/paper-apps,673,bluebirdfirewin,2025-02-06T07:58:33 1738828713,94,2025-02-06
4,5,OpenLDK: A Java JIT compiler and runtime in Co...,https://github.com/atgreen/openldk,212,varjag,2025-02-05T12:17:47 1738757867,61,2025-02-06
...,...,...,...,...,...,...,...,...
2235,146,Show HN: Tired of building agents? throw an LL...,https://github.com/The-Pocket-World/Pocket-Flo...,14,hez2000,2025-02-19T23:55:34 1740009334,5,2025-02-20
2236,147,These Are the SpaceX Engineers Working Inside ...,https://www.wired.com/story/faa-doge-elon-musk...,13,rntn,2025-02-20T16:15:16 1740068116,3,2025-02-20
2237,148,"The Incompetence of Doge Is a Feature, Not a Bug",https://www.wired.com/story/doge-incompetence-...,12,xnx,2025-02-20T14:41:16 1740062476,1,2025-02-20
2238,149,Federal Trade Commission Launches Inquiry on T...,https://www.ftc.gov/news-events/news/press-rel...,11,gnabgib,2025-02-20T17:49:40 1740073780,2,2025-02-20


In [7]:
import pandas as pd
from mistralai import Mistral

client = Mistral(api_key=MISTRAL_API_KEY)
model = "mistral-embed"

def get_embeddings_by_chunks(data, chunk_size):
    chunks = [data[x : x + chunk_size] for x in range(0, len(data), chunk_size)]
    embeddings_response = [
        client.embeddings.create(model=model, inputs=c) for c in chunks
    ]
    return [d.embedding for e in embeddings_response for d in e.data]

df["embeddings"] = get_embeddings_by_chunks(df["title"].tolist(), 50)
df.head()

,rank,title,url,points,author,created_at,comments,partition_date,embeddings
0,1,Simulating water over terrain,https://lisyarus.github.io/blog/posts/simulati...,387,ibobev,2025-02-06T14:15:28 1738851328,58,2025-02-06,"[-0.005764007568359375, 0.0252685546875, 0.030..."
1,2,Programming SDF animations of Rick and Morty,https://danielchasehooper.com/posts/code-anima...,505,LordNibbler,2025-02-06T03:30:34 1738812634,39,2025-02-06,"[-0.002887725830078125, 0.015716552734375, 0.0..."
2,3,Show HN: SQLite disk page explorer,https://github.com/QuadrupleA/sqlite-page-expl...,288,QuadrupleA,2025-02-06T18:40:30 1738867230,41,2025-02-06,"[0.0167999267578125, 0.013885498046875, 0.0329..."
3,4,Paper Apps,https://gladdendesign.com/collections/paper-apps,673,bluebirdfirewin,2025-02-06T07:58:33 1738828713,94,2025-02-06,"[-0.01995849609375, 0.036651611328125, 0.03616..."
4,5,OpenLDK: A Java JIT compiler and runtime in Co...,https://github.com/atgreen/openldk,212,varjag,2025-02-05T12:17:47 1738757867,61,2025-02-06,"[-0.01100921630859375, 0.03118896484375, 0.035..."


In [14]:
from sklearn.metrics.pairwise import euclidean_distances

def search(term):
    pass

In [45]:
def search(df, term):
    df = df.copy()
    response = client.embeddings.create(model=model, inputs=[term])
    term_embedding = response.data[0].embedding

    df["distance"] = df["embeddings"].apply(lambda x: euclidean_distances([x], [term_embedding])[0][0])

    return df.sort_values("distance")[["title", "distance"]][:5]

In [46]:
search(df, "elon musk")

,title,distance
1785,Elon Musk's A.I.-Fuelled War on Human Agency,0.676006
1584,Elon Musk's Toxicity Could Spell Disaster for ...,0.686422
1710,Elon Musk's terrifying vision for AI,0.698069
2205,How Americans View Elon Musk and Mark Zuckerberg,0.710151
2033,Elon Musk is failing to cut American spending,0.719528
